In [ ]:
### 필요 함수 정의

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
### 필수 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

import copy
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, plot_metric
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import optuna
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, f1_score, precision_recall_curve
import seaborn as sns
import warnings;warnings.filterwarnings('ignore')
import eli5
from eli5.sklearn import PermutationImportance

# regression for stacking model
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import RobustScaler
        
# ensemble
from sklearn.ensemble import VotingClassifier,  RandomForestClassifier, StackingClassifier, ExtraTreesClassifier

In [ ]:
### 데이터셋 읽어오기

In [ ]:
df_train = pd.read_csv('new_imputation_train_ddd.csv')
df_test = pd.read_csv('new_imputation_test_ddd.csv')

In [ ]:
# 데이터 추가 처리

In [ ]:
# self label encoding
con = [
    df_train['expected_timeline'] == 'others',
    df_train['expected_timeline'] == 'no_answer',
    df_train['expected_timeline'] == 'less_than_3_months',
    df_train['expected_timeline'] == '3_months_6_months',
    df_train['expected_timeline'] == '6_months_9_months',
    df_train['expected_timeline'] == '9_months_1_year'
]
cri = [
    -1, 0, 1, 2, 2, 2
]
df_train['timeline_label'] = np.select(con, cri, default = 3)

In [ ]:
# self label encoding
con = [
    df_test['expected_timeline'] == 'others',
    df_test['expected_timeline'] == 'no_answer',
    df_test['expected_timeline'] == 'less_than_3_months',
    df_test['expected_timeline'] == '3_months_6_months',
    df_test['expected_timeline'] == '6_months_9_months',
    df_test['expected_timeline'] == '9_months_1_year'
]
cri = [
    -1, 0, 1, 2, 2, 2
]
df_test['timeline_label'] = np.select(con, cri, default = 3)

In [ ]:
df_train['bant_submit'] = (df_train['bant_submit']*4).astype(int)
df_test['bant_submit'] = (df_test['bant_submit']*4).astype(int)

In [ ]:
# business_unit CM을 IT에 포함
# CM이 2건밖에 없으므로 가장 관련있는 부서와 함께 묶어넣기
df_train['business_unit'][df_train['business_unit'] == 'CM'] = 'IT'
df_test['business_unit'][df_test['business_unit'] == 'CM'] = 'IT'

In [ ]:
# 개수가 적은 값을 others로 같이 넣기 -> 학습 시켜서 분류하는데 도음이 되지 않음
# 20개 이하는 모조리 같이 묶어버리자
choose_low_category = [
    'customer_country', 'customer_type', 'customer_job', 'inquiry_type',
    'product_category', 'business_subarea'
]

other_name = [
    'Others', 'Other', 'Others', 'others', 'others', 'Others'
]

cnt = [
    10, 10, 10, 10, 10, 10
]

for c, i in enumerate(choose_low_category):
    frequency = df_train[i].value_counts()
    low_frequency_categories = frequency[frequency <= cnt[c]].index
    df_train[i] = df_train[i].apply(lambda x: other_name[c] if x in low_frequency_categories else x)
    df_test[i] = df_test[i].apply(lambda x: other_name[c] if x in low_frequency_categories else x)

In [ ]:
# ver_win_ratio_per_bu
data_merge1 = (df_train.groupby(['business_area', 'business_unit']).sum()['is_converted']/df_train.groupby(['business_area', 'business_unit']).count()['is_converted']).reset_index()
data_merge1.columns = ['business_area', 'business_unit', 'ver_win_ratio_per_bu']

# com_reg_ver_win_rate
data_merge2 = (df_train.groupby(['business_area', 'business_unit', 'continent']).sum()['is_converted']/df_train.groupby(['business_area', 'business_unit', 'continent']).count()['is_converted']).reset_index()
data_merge2.columns = ['business_area', 'business_unit', 'continent', 'com_reg_ver_win_rate']

# ver_win_rate_x
a1 = df_train['business_area'].value_counts()/len(df_train)
a2 = df_train.groupby(['business_area'])['is_converted'].mean()
data_merge3 = (a1 * a2).reset_index()
data_merge3.columns = ['business_area', 'ver_win_rate_x']

In [ ]:
# `bant_submit`과 `expected_timeline`을 기준으로 그룹화한 후, 각 그룹별 전환율 계산
grouped = df_train.groupby(['bant_submit', 'expected_timeline']).agg(
    conversions=('is_converted', 'sum'),
    total=('is_converted', 'count')
)
grouped['conversion_rate'] = grouped['conversions'] / grouped['total']

# 전환율을 데이터프레임으로 리셋
conversion_rates = grouped.reset_index()[['bant_submit', 'expected_timeline', 'conversion_rate']]

# 훈련 및 테스트 데이터셋에 전환율 정보를 병합
df_train = pd.merge(df_train, conversion_rates, on=['bant_submit', 'expected_timeline'], how='left')
df_test = pd.merge(df_test, conversion_rates, on=['bant_submit', 'expected_timeline'], how='left')

In [ ]:
# 결합할 데이터 새로 끼워넣기 위해 
df_train.drop(['ver_win_ratio_per_bu', 'com_reg_ver_win_rate', 'ver_win_rate_x', 'expected_timeline'], axis = 1, inplace = True)
df_test.drop(['ver_win_ratio_per_bu', 'com_reg_ver_win_rate', 'ver_win_rate_x', 'expected_timeline'], axis = 1, inplace = True)

In [ ]:
# 하나씩 결합 고
tmp = pd.merge(df_train, data_merge1, how = 'left', on = ['business_area', 'business_unit'])
tmp2 = pd.merge(tmp, data_merge2, how = 'left', on = ['business_area', 'business_unit', 'continent'])
df_train = pd.merge(tmp2, data_merge3, how = 'left', on = ['business_area'])

# 하나씩 결합 고
tmp = pd.merge(df_test, data_merge1, how = 'left', on = ['business_area', 'business_unit'])
tmp2 = pd.merge(tmp, data_merge2, how = 'left', on = ['business_area', 'business_unit', 'continent'])
df_test = pd.merge(tmp2, data_merge3, how = 'left', on = ['business_area'])

In [ ]:
### Label Encoding

In [ ]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [ ]:
# 레이블 인코딩할 칼럼들
label_columns = [
    'customer_country',
    'business_unit',
    'customer_type',
    'enterprise',
    'customer_job',
    'inquiry_type',
    'product_category',
    'response_corporate',
    'business_area',
    'business_subarea',
    'lead_owner',
    'continent'
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
    
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [ ]:
### 직접 계산하기

In [ ]:
# # 0을 근사값으로 대치
# df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']] = df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']].replace(0, 1e-12)
# df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']] = df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']].replace(0, 1e-12)

# df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']].replace(1, 1 - 1e-12)
# df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']].replace(1, 1 - 1e-12)

In [ ]:
# # logit으로 범위 변형
# df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = np.log(df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] / (1 - df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']]))
# df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = np.log(df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] / (1 - df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']]))

In [ ]:
# # log로 범위 변형
# df_train['historical_existing_cnt'] = np.log(df_train['historical_existing_cnt'])
# df_test['historical_existing_cnt'] = np.log(df_test['historical_existing_cnt'])

In [ ]:
# # IterativeImputer 객체 생성
# imputer = IterativeImputer(max_iter=10, random_state=42)  # 적절한 max_iter 값 설정

# # 데이터에 결측치 대치
# df_train[df_train.columns.difference(['is_converted'])] = imputer.fit_transform(df_train[df_train.columns.difference(['is_converted'])])

# # test
# df_test[df_test.columns.difference(['id', 'is_converted'])] = imputer.transform(df_test[df_test.columns.difference(['id', 'is_converted'])])

In [ ]:
# # 로짓의 역변환
# df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = 1 / (1 + np.exp(-df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']]))
# df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = 1 / (1 + np.exp(-df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']]))

In [ ]:
# # 지수변환
# df_train['historical_existing_cnt'] = np.exp(df_train['historical_existing_cnt'])
# df_test['historical_existing_cnt'] = np.exp(df_test['historical_existing_cnt'])

In [ ]:
# # 근사값 원래대로 돌리기
# df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']] = df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']].replace(1e-12, 0)
# df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']] = df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']].replace(1e-12, 0)

# df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = df_train[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']].replace(1 - 1e-12, 1)
# df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']] = df_test[['ver_win_ratio_per_bu', 'ver_win_rate_x', 'com_reg_ver_win_rate']].replace(1 - 1e-12, 1)

In [ ]:
### 변수 이상한것들 다 쳐내

In [ ]:
categorical_features = [
    'bant_submit',
    'customer_country',
    'business_unit',
    'customer_idx',
    'customer_type',
    'enterprise',
    'id_strategic_ver',
    'it_strategic_ver',
    'idit_strategic_ver',
    'customer_job',
    'inquiry_type',
    'product_category',
    'response_corporate',
    'ver_cus',
    'ver_pro',
    'business_area',
    'business_subarea',
    'lead_owner',
    'continent',
    'fan_mae_king_yn',
    'country_to_product',
    'partnership',
    'move_corporate'
]

In [ ]:
# historical_existing_cnt
df_train['historical_existing_cnt'] = df_train['historical_existing_cnt'].fillna(0)
df_test['historical_existing_cnt'] = df_test['historical_existing_cnt'].fillna(0)

In [ ]:
# 0, 1로 되는건 전부 int형으로 바꿔줘야 함
for i in categorical_features:
    if df_train[i].dtypes == 'float64':
        df_train[i] = df_train[i].astype('int64')
        df_test[i] = df_test[i].astype('int64')
        
# 추가적으로 int로 바꿔줘야 하는것들 다 바꾸기
df_train['historical_existing_cnt'] = df_train['historical_existing_cnt'].astype('int64')
df_test['historical_existing_cnt'] = df_test['historical_existing_cnt'].astype('int64')

df_train['lead_desc_length'] = df_train['lead_desc_length'].astype('int64')
df_test['lead_desc_length'] = df_test['lead_desc_length'].astype('int64')

df_train['timeline_label'] = df_train['timeline_label'].astype('int64')
df_test['timeline_label'] = df_test['timeline_label'].astype('int64')

In [ ]:
# is_converted 제외하고 모두 같은값이지만 두개의 값 전부 가지고 있음
# True2 False1이므로 강제로 True로 바꿔주기
# 나머지는 True, False 통일되어있음
df_train['is_converted'][df_train.index == 31692] = True

In [ ]:
# 모델링 준비

In [ ]:
### 학습 데이터 분리

In [ ]:
# # 마지막으로 버릴 변수 버리기
# df_train.drop(['business_area', 'idit_strategic_ver', 'historical_existing_cnt', 'lead_desc_length', 'bant_submit'], axis = 1, inplace = True)
# df_test.drop(['business_area', 'idit_strategic_ver', 'historical_existing_cnt', 'lead_desc_length', 'bant_submit'], axis = 1, inplace = True)

In [ ]:
# 숫자형 robust scaler
rob = RobustScaler()

In [ ]:
# 적용시킬 변수 가져오기
numeric_features = []
for j, k in zip(df_train.dtypes, df_train.columns):
    if j == 'float64' or k in ['historical_existing_cnt', 'lead_desc_length']:
        numeric_features.append(k)

In [ ]:
# robust
df_train[numeric_features] = rob.fit_transform(df_train[numeric_features])
df_test[numeric_features] = rob.transform(df_test[numeric_features])

In [ ]:
X = df_train.drop("is_converted", axis=1)
y = df_train['is_converted']

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.1,
    shuffle=True,
    stratify = y,
    random_state=42,
)

In [ ]:
# smote = SMOTE(sampling_strategy='auto', random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
for cat_feature in categorical_features:
    if cat_feature in X_train.columns:
        X_train[cat_feature] = X_train[cat_feature].astype('category')
        X_val[cat_feature] = X_val[cat_feature].astype('category')

In [ ]:
df_train.describe()

In [ ]:
# 모델 학습

In [ ]:
# f1 score
f1_scorer = make_scorer(f1_score, pos_label=1, average = 'binary')

In [ ]:
### n_trials 공통으로 정하기
n_trial = 30
seed = 42
n_split = 10

In [ ]:
### 1. LGBM

In [ ]:
def objective(trial):
    params = {
        'boosting_type' : trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'rf']),
        'n_estimators' : trial.suggest_int('n_estimators', 100, 1000),
        "max_depth":trial.suggest_int('max_depth', 3, 10),
        "learning_rate" : trial.suggest_float('learning_rate',1e-2, 0.1, log=True), 
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10, log=True),
        "min_child_weight" : trial.suggest_float('min_child_weight', 1, 20),
        "min_child_samples" : trial.suggest_int('min_child_samples', 1, 50),
        "subsample" : trial.suggest_float('subsample', 0.6, 0.9),
        "subsample_freq" : trial.suggest_int('subsample_freq', 0, 5),
        "colsample_bytree" : trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 64),
        'n_jobs' : -1,
        'class_weight' : {True: 9, False: 1}
    }

    model = LGBMClassifier(**params, device = 'cpu', random_state=seed, verbose=-1)

    # KFold : 회귀모델 / StratifiedKFold : 분류모델
    skf = StratifiedKFold(n_splits=n_split, shuffle=True, random_state=seed)
    cv = cross_val_score(model, X_train, y_train, scoring=f1_scorer,cv=skf).mean()
    
    return cv

In [ ]:
%%time
study_lgbm = optuna.create_study(direction="maximize")
study_lgbm.optimize(objective, n_trials=n_trial)

In [ ]:
print('Best trial: score {}, \nparams {}'.format(study_lgbm.best_trial.value,
                                                 study_lgbm.best_trial.params))

# Best trial: score 0.86819626880691, 
# params {'boosting_type': 'gbdt', 'n_estimators': 757, 'max_depth': 9, 'learning_rate': 0.059966739514413524, 'reg_alpha': 0.037751355862792976, 'reg_lambda': 0.00010263802073907619, 'min_child_weight': 11.59612675536348, 'min_child_samples': 16, 'subsample': 0.7646796348565887, 'subsample_freq': 1, 'colsample_bytree': 0.8673855331452011, 'num_leaves': 49}

In [ ]:
### 학습

In [ ]:
best_params_lgbm = study_lgbm.best_params
best_model_lgbm = LGBMClassifier(**best_params_lgbm, random_state=seed)
best_model_lgbm.fit(X_train, y_train)

In [ ]:
# 위 feature importance를 시각화해봅니다.
importances_lgbm = pd.Series(best_model_lgbm.feature_importances_, index=list(X_train.columns))
importances_lgbm = importances_lgbm.sort_values(ascending=False)

plt.figure(figsize=(10,8))
plt.title("Feature Importances")
sns.barplot(x=importances_lgbm, y=importances_lgbm.index)
plt.show()

In [ ]:
y_pred_proba_lgbm = best_model_lgbm.predict_proba(X_val)[:, 1]

In [ ]:
fpr_lgbm, tpr_lgbm, thresholds_lgbm = roc_curve(y_val, y_pred_proba_lgbm)

In [ ]:
# ROC 곡선 그리기
plt.figure()
plt.plot(fpr_lgbm, tpr_lgbm, label='ROC curve (area = %0.2f)' % auc(fpr_lgbm, tpr_lgbm))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# 적정 임계값 찾기
optimal_idx_lgbm = np.argmax(tpr_lgbm - fpr_lgbm)
optimal_threshold_lgbm = thresholds_lgbm[optimal_idx_lgbm]
print("Optimal threshold:", optimal_threshold_lgbm)

In [ ]:
y_pred_custom_threshold_lgbm = (y_pred_proba_lgbm >= optimal_threshold_lgbm).astype(int)

In [ ]:
### 기본 쓰레스홀드
pred_lgbm = best_model_lgbm.predict(X_val)
get_clf_eval(y_val, pred_lgbm)

In [ ]:
### 최적 쓰레스홀드
get_clf_eval(y_val, y_pred_custom_threshold_lgbm)

In [ ]:
### 2. ET

In [ ]:
# optuna 학습 메소드 선언
def objective_ET(trial, X, y):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.2),
        'class_weight' : {True: 9, False: 1}
    }
    model = ExtraTreesClassifier(**param)
    
    # KFold : 회귀모델 / StratifiedKFold : 분류모델
    skf = StratifiedKFold(n_splits=n_split, shuffle=True, random_state=seed)
    cv = cross_val_score(model, X, y, scoring=f1_scorer, cv=skf).mean()
    
    return cv

In [ ]:
%%time
study_et = optuna.create_study(study_name='ET', direction="maximize")
study_et.optimize(lambda trial: objective_ET(trial, X_train, y_train),
                  n_trials = n_trial)

In [ ]:
# print하자
print('Best trial: score {}, \nparams {}'.format(study_et.best_trial.value,
                                                 study_et.best_trial.params))

In [ ]:
# best models for rf
best_params_et = study_et.best_params
best_model_et = ExtraTreesClassifier(**best_params_et, random_state=42)
best_model_et.fit(X_train, y_train)

In [ ]:
# 위 feature importance를 시각화해봅니다.
importances_et = pd.Series(best_model_et.feature_importances_, index=list(X_train.columns))
importances_et = importances_et.sort_values(ascending=False)

plt.figure(figsize=(10,8))
plt.title("Feature Importances")
sns.barplot(x=importances_et, y=importances_et.index)
plt.show()

In [ ]:
y_pred_proba_et = best_model_et.predict_proba(X_val)[:, 1]

In [ ]:
fpr_et, tpr_et, thresholds_et = roc_curve(y_val, y_pred_proba_et)

In [ ]:
# ROC 곡선 그리기
plt.figure()
plt.plot(fpr_et, tpr_et, label='ROC curve (area = %0.2f)' % auc(fpr_et, tpr_et))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# 적정 임계값 찾기
optimal_idx_et = np.argmax(tpr_et - fpr_et)
optimal_threshold_et = thresholds_et[optimal_idx_et]
print("Optimal threshold:", optimal_threshold_et)

In [ ]:
y_pred_custom_threshold_et = (y_pred_proba_et >= optimal_threshold_et).astype(int)

In [ ]:
### 기본 쓰레스홀드
pred_et = best_model_et.predict(X_val)
get_clf_eval(y_val, pred_et)

In [ ]:
### 최적 쓰레스홀드
get_clf_eval(y_val, y_pred_custom_threshold_et)

In [ ]:
### 3. Soft Voting

In [ ]:
# soft voting
soft_voting_model = VotingClassifier(estimators=[
                                                 ('Extra Tree', best_model_et),
                                                 ('Light GBM', best_model_lgbm)
                                                ],
                                     voting='soft',
                                     weights=[1, 1])

In [ ]:
soft_voting_model.fit(X_train, y_train)

In [ ]:
# predict & confusion Matrix
pred_soft_voting = soft_voting_model.predict(X_val)
get_clf_eval(y_val, pred_soft_voting)

In [ ]:
y_pred_proba_sv = soft_voting_model.predict_proba(X_val)[:, 1]

In [ ]:
fpr_sv, tpr_sv, thresholds_sv = roc_curve(y_val, y_pred_proba_sv)

In [ ]:
# ROC 곡선 그리기
plt.figure()
plt.plot(fpr_sv, tpr_sv, label='ROC curve (area = %0.2f)' % auc(fpr_sv, tpr_sv))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# 적정 임계값 찾기
optimal_idx_sv = np.argmax(tpr_sv - fpr_sv)
optimal_threshold_sv = thresholds_sv[optimal_idx_sv]
print("Optimal threshold:", optimal_threshold_sv)

In [ ]:
y_pred_custom_threshold_sv = (y_pred_proba_sv >= optimal_threshold_sv).astype(int)

In [ ]:
### 기본 쓰레스홀드
pred_sv = soft_voting_model.predict(X_val)
get_clf_eval(y_val, pred_sv)

In [ ]:
### 최적 쓰레스홀드
get_clf_eval(y_val, y_pred_custom_threshold_sv)

In [ ]:
### 4. Stacking

In [ ]:
estimators=[
    ('Extra Tree', best_model_et),
    ('Light GBM', best_model_lgbm),
    ('Soft_voting', soft_voting_model)
]
stacking_model = StackingClassifier(
     estimators=estimators,
     final_estimator=LogisticRegression()
)

In [ ]:
stacking_model.fit(X_train, y_train)

In [ ]:
# predict & confusion Matrix
pred_stacking = stacking_model.predict(X_val)
get_clf_eval(y_val, pred_stacking)

In [ ]:
y_pred_proba_stack = stacking_model.predict_proba(X_val)[:, 1]

In [ ]:
fpr_stack, tpr_stack, thresholds_stack = roc_curve(y_val, y_pred_proba_stack)

In [ ]:
# ROC 곡선 그리기
plt.figure()
plt.plot(fpr_stack, tpr_stack, label='ROC curve (area = %0.2f)' % auc(fpr_stack, tpr_stack))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# 적정 임계값 찾기
optimal_idx_stack = np.argmax(tpr_stack - fpr_stack)
optimal_threshold_stack = thresholds_stack[optimal_idx_stack]
print("Optimal threshold:", optimal_threshold_stack)

In [ ]:
y_pred_custom_threshold_stack = (y_pred_proba_stack >= optimal_threshold_stack).astype(int)

In [ ]:
### 기본 쓰레스홀드
pred_stack = stacking_model.predict(X_val)
get_clf_eval(y_val, pred_stack)

In [ ]:
### 최적 쓰레스홀드
get_clf_eval(y_val, y_pred_custom_threshold_stack)

In [ ]:
# 제출용 데이터 만들기

In [ ]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [ ]:
X_merged = pd.concat([X_train, X_val])
y_merged = pd.concat([y_train, y_val])

In [ ]:
# params
lgbm_params = best_params_lgbm

In [ ]:
# threshold
optimal_threshold_final = optimal_threshold_lgbm

In [ ]:
y.value_counts()/y.count()

In [ ]:
for cat_feature in categorical_features:
    if cat_feature in X_train.columns:
        X[cat_feature] = X[cat_feature].astype('category')
        X_merged[cat_feature] = X_merged[cat_feature].astype('category')
        x_test[cat_feature] = x_test[cat_feature].astype('category')

In [ ]:
# soft voting
final_model = LGBMClassifier(
#  **best_params,
    **lgbm_params,
    random_state=42
)

final_pred = final_model.fit(X, y).predict(x_test)
sum(final_pred) # True로 예측된 개수

In [ ]:
# lgbm_params = best_params

In [ ]:
# # best_params = study.best_params
# best_model_lgbm = LGBMClassifier(
# #                            **best_params,
#                            **lgbm_params,
#                            random_state=1996)

# test_pred_lgbm = best_model_lgbm.fit(X,y).predict(x_test)
# sum(test_pred_lgbm) # True로 예측된 개수

In [ ]:
# y_pred_proba_lgbm = best_model_lgbm.predict_proba(x_test)[:, 1]  # 양성 클래스에 대한 확률
# y_pred_custom_threshold_lgbm = (y_pred_proba_lgbm >= optimal_threshold_lgbm).astype(int)

In [ ]:
y_pred_proba_final = final_model.predict_proba(x_test)[:, 1]  # 양성 클래스에 대한 확률
y_pred_custom_threshold_final = (y_pred_proba_final >= optimal_threshold_final).astype(int)

In [ ]:
sum(y_pred_custom_threshold_final)

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = y_pred_custom_threshold_final

# 제출 파일 저장
df_sub.to_csv("submission_tmp002.csv", index=False)